In [4]:
pip install caer canaro

In [24]:
import os 
import caer
import canaro
import numpy as np
import cv2 as cv
import gc # garbage collector

In [25]:
IMG_SIZE = (80,80)
channels = 1 # nu avem nevoie de imaginea color, vom avea nevoie doar de un canal
char_path = r'../input/the-simpsons-characters-dataset/simpsons_dataset'

In [26]:
char_dict = {}
for char in os.listdir(char_path):
    char_dict[char] = len(os.listdir(os.path.join(char_path,char))) # mergem prin fiecare folder si luam numele acestuia si numarul de poze din el

# Sortam descrescator dictionarul
char_dict = caer.sort_dict(char_dict, descending = True)
char_dict

In [27]:
characters = []
count = 0
for i in char_dict:
    count += 1
    characters.append(i[0])
    if count >= 10:
        break
characters

In [28]:
# Creem training data

train = caer.preprocess_from_dir(char_path, characters, channels = channels, IMG_SIZE = IMG_SIZE, isShuffle = True) 
# Cauta in fiecare folder , characters si cand gaseste unu caracter din lista characters ia toate imaginile din acesta

In [29]:
len(train)

In [30]:
import matplotlib.pyplot as plt
plt.figure(figsize=(30,30))
plt.imshow(train[0][0],cmap='gray')
plt.show()


In [31]:
featureSet, labels = caer.sep_train(train, IMG_SIZE=IMG_SIZE)


In [32]:
from tensorflow.keras.utils import to_categorical
# Normalizam feature set ==> (0,1)
featureSet = caer.normalize(featureSet)
labels = to_categorical(labels,len(characters))
labels

In [33]:
x_train, x_val, y_train, y_val = caer.train_val_split(featureSet, labels, val_ratio=.2) # 20% vor merge la validation set si restul la training set

In [34]:
# Pentru a salva memorie putem sa stergem unele variabile pe care nu le mai folosim mai departe
del train
del featureSet
del labels
gc.collect() # colectam in garbage collector

In [103]:
BATCH_SIZE = 2
EPOCHS = 50

In [104]:
# Creeam un image data generator
datagen = canaro.generators.imageDataGenerator() # instanteaza un image generator simplu
train_gen = datagen.flow(x_train, y_train, batch_size = BATCH_SIZE)

In [107]:
# Crearea modelului
model = canaro.models.createSimpsonsModel(IMG_SIZE=IMG_SIZE, channels = channels, output_dim = len(characters),
                                         loss ='binary_crossentropy', decay = 1e-6, learning_rate = 0.001, momentum = 0.9,
                                         nesterov = True)

In [108]:
model.summary()

In [109]:
from tensorflow.keras.callbacks import LearningRateScheduler
# Crearea unui call-back-list , acesta va contine un learning_rate schedule care va programara la un anumit timp learning_rate pentru o antrenare mai buna
callbacks_list = [LearningRateScheduler(canaro.lr_schedule)]

In [110]:
training = model.fit(train_gen, steps_per_epoch = len(x_train)// BATCH_SIZE,
                    epochs = EPOCHS,
                    validation_data = (x_val, y_val),
                    validation_steps= len(y_val)//BATCH_SIZE,
                     callbacks = callbacks_list)

In [98]:
test_path = r'../input/the-simpsons-characters-dataset/kaggle_simpson_testset/kaggle_simpson_testset/chief_wiggum_20.jpg'

img = cv.imread(test_path)
plt.imshow(img)
plt.show()



In [99]:
# preparam imaginea pentru a fi de acelasi size si shape ca si imaginile pentru care a fost facuta antrenarea
def prepare(img):
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img = cv.resize(img, IMG_SIZE)
    img = caer.reshape(img, IMG_SIZE, 1)    
    return img

In [100]:
prediction = model.predict(prepare(img))

In [101]:
prediction

In [102]:
print(characters[np.argmax(prediction[0])])